In [1]:
import numpy as np
import torch as th


from glob import glob
from utils import *
import ray
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
ray.init(num_cpus=160, ignore_reinit_error=True)

2023-08-31 09:24:44,601	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.11.3
Ray version:,2.6.2
Dashboard:,http://127.0.0.1:8265


In [3]:
insmp = Backtester(20190103, 20210101, pattern = "./checkpoints/*sat*/",)
otsmp = Backtester(20210101, 20211231, pattern = "./checkpoints/*sat*/",)

num of factors:191
num of factors:191


In [4]:
factors = insmp.calc_factor()
factors2 = otsmp.calc_factor()

calc_factor took 334 seconds
calc_factor took 193 seconds


In [5]:
df_ic = insmp.calc_ic()
df_ic2 = otsmp.calc_ic()


calc_ic took 906 seconds
calc_ic took 492 seconds


In [6]:
ic_filter_idx = otsmp.df_ic.query("y1>0.01 and y2>0.01 and y5>0.01 and @insmp.df_ic.y1>.01 and @insmp.df_ic.y2>.015 and @insmp.df_ic.y5>.02").index

In [7]:
feature_dict = dict(zip(otsmp.factor_names,otsmp.factors))

In [8]:
ic_filter_sigs = [feature_dict[i] for i in ic_filter_idx]

In [9]:
# def calc_sig_corr(sigs):
#     n = len(sigs)
#     df_corr = np.ones((n,n))
#     for i in range(n):
#         for j in range(i+1,n):
#             ic = torch.nanmean(batch_pearsonr(sigs[i],sigs[j],))
#             df_corr[i,j] = ic
#             df_corr[j,i] = ic
#     return df_corr
from concurrent.futures import ProcessPoolExecutor
            
def calc_single_corr(i, j, sigs):
    return torch.nanmean(batch_pearsonr(sigs[i], sigs[j]))

def calc_sig_corr(sigs):
    n = len(sigs)
    df_corr = np.ones((n, n))
    
    with ProcessPoolExecutor() as executor:
        future_to_index = {(i, j): executor.submit(calc_single_corr, i, j, sigs) for i in range(n) for j in range(i+1, n)}
        
        for (i, j), future in future_to_index.items():
            ic = future.result()
            df_corr[i, j] = ic
            df_corr[j, i] = ic
            
    return df_corr

In [10]:
df_corr = calc_sig_corr(ic_filter_sigs)

In [11]:
df_corr = pd.DataFrame(df_corr)

In [12]:
lowcorr_idx = remove_high_corr_factors(df_corr.values)
print(len(lowcorr_idx))

101


In [13]:
final = [ic_filter_idx[idx] for idx in lowcorr_idx]

In [14]:

# 设置显示最大列数
pd.set_option('display.max_columns', None)

# 设置显示最大行数
pd.set_option('display.max_rows', None)
finaldf =  otsmp.df_ic.loc[final]

In [15]:
finaldf.describe()

,y1,y2,y5
count,101.000000,101.000000,101.000000
mean,0.022701,0.024558,0.027732
std,0.006382,0.006631,0.008423
min,0.010131,0.011148,0.011538
25%,0.017732,0.019231,0.020554
50%,0.022777,0.024574,0.028378
75%,0.026686,0.029093,0.034180
max,0.036356,0.039204,0.047202


In [16]:
fs  = ic_filter_sigs
y1d = otsmp.calter.ret1d
y2d = otsmp.calter.ret2d
y5d = otsmp.calter.ret5d

In [18]:
finaldf

,y1,y2,y5
20230811171800_satd_lexpr08_lopt34_10_0042,0.029813,0.032791,0.039772
20230813143935_satd_lexpr08_lopt34_10_0322,0.011760,0.013922,0.012883
20230813145451_satd_lexpr08_lopt34_10_0856,0.026628,0.030035,0.037093
20230813145501_satd_lexpr08_lopt34_10_0726,0.029509,0.029523,0.031023
20230814092613_satd_lexpr08_lopt34_10_0788,0.030111,0.031117,0.033647
20230814092614_satd_lexpr08_lopt34_10_0177,0.010296,0.011239,0.015462
20230814092615_satd_lexpr08_lopt34_10_0018,0.021988,0.023256,0.023869
20230814092615_satd_lexpr08_lopt34_10_0143,0.022127,0.025572,0.027143
20230814102408_satd_lexpr15_lopt34_10_0319,0.026133,0.027363,0.035816
20230814175115_satd_lexpr15_lopt34_10_0852,0.035036,0.036162,0.041299


In [19]:
finaldf.to_csv('result.csv')

In [20]:
df_corr.iloc[lowcorr_idx,lowcorr_idx].to_csv('df_corr.csv')

In [21]:
df_corr.replace(1,0).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139
count,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000
mean,0.377814,0.347505,0.520920,0.524005,0.317964,0.440907,0.230506,0.474328,0.362150,0.386490,0.296263,0.515116,0.515264,0.549672,0.522633,0.379740,0.501493,0.389613,0.535048,0.490115,0.537551,0.562798,0.428700,0.337485,0.364189,0.515480,0.501552,0.502193,0.135730,0.529216,0.561646,0.447295,0.323276,0.399020,0.553136,0.521365,0.438641,0.569855,0.189752,0.404913,0.519624,0.379141,0.556952,0.558875,0.339577,0.363233,0.528522,0.429411,0.456735,0.546688,0.360192,0.476008,0.421493,0.284771,0.406611,0.389152,0.418271,0.449185,0.473633,0.360422,0.322176,0.351688,0.414560,0.457104,0.552077,0.430478,0.383709,0.522724,0.239261,0.390510,0.184903,0.431370,0.365029,0.186489,0.548620,0.211885,0.553526,0.381589,0.342311,0.149248,0.453541,0.534118,0.426771,0.562608,0.408372,0.300636,0.211688,0.231470,0.404540,0.421905,0.433644,0.481424,0.404120,0.316378,0.549183,0.342563,0.413128,0.509917,0.548623,0.339885,0.460154,0.444518,0.204878,0.516738,0.449922,0.343887,0.409316,0.292840,0.550738,0.522709,0.397421,0.193120,0.468284,0.530761,0.462230,0.375327,0.472977,0.337849,0.388548,0.384392,0.355743,0.532485,0.398781,0.201852,0.266361,0.382589,0.382948,0.499910,0.499706,0.537153,0.508530,0.546476,0.536069,0.417790,0.577631,0.474112,0.343908,0.161587,0.406132,0.302049
std,0.184824,0.151472,0.165883,0.180372,0.104227,0.149042,0.188458,0.156345,0.146948,0.115642,0.138545,0.202401,0.169059,0.201811,0.159805,0.168971,0.168604,0.134691,0.178597,0.176052,0.189990,0.170999,0.135543,0.125335,0.139849,0.187877,0.140277,0.167580,0.102794,0.183295,0.178524,0.142187,0.181825,0.136648,0.183986,0.163056,0.147891,0.171714,0.075169,0.171702,0.211219,0.134067,0.176580,0.184100,0.127048,0.114973,0.180513,0.156689,0.135260,0.195148,0.164490,0.140303,0.140203,0.113377,0.133725,0.143312,0.138998,0.175865,0.156249,0.149639,0.167775,0.126324,0.155287,0.169062,0.171074,0.183953,0.131101,0.186286,0.097676,0.112562,0.095148,0.137316,0.155192,0.094693,0.203756,0.099435,0.193332,0.188291,0.101474,0.062937,0.163849,0.182